 <img src="smart-building.drawio.png" />


In [1]:
import time
from pprint import pprint
from tqdm import tqdm

#avenieca sdk imports
from avenieca import Signal
from avenieca.producers import Event
from avenieca.api.model import *
from avenieca.api.eca import ECA

#local file imports
from config import *
from util import *

def prettyprint(res, status):
    try:
        pprint(res.__dict__)
    except:
        print(res)
    print(status)

load_dotenv()
data_path = os.getenv("DATA_PATH")
url = '%s/iot_data.csv' % data_path

In [2]:
import pandas as pd
data = pd.read_csv(url)

### Stream to ECA Twins

In [3]:
aqi_broker_config = aqi_twin_config.broker_config
aqi_event = Event(config=aqi_broker_config)

temperature_broker_config = temperature_twin_config.broker_config
temperature_event = Event(config=temperature_broker_config)

occupancy_broker_config = occupancy_twin_config.broker_config
occupancy_event = Event(config=occupancy_broker_config)

ac_broker_config = ac_twin_config.broker_config
ac_event = Event(config=ac_broker_config)

purifier_broker_config = purifier_twin_config.broker_config
purifier_event = Event(config=purifier_broker_config)

aqi_data = data["Air Quality Index"].values
temp_data = data["Temperature"].values
occupancy_data = data["Occupancy"].values
ac_data = data["Air Conditioner"].values
purifier_data = data["Purifier"].values

for i in tqdm(range(0, len(aqi_data))):
    aqi_signal = Signal(
        state=[float(aqi_data[i])]
    )
    
    temp_signal = Signal(
        state=[float(temp_data[i])]
    )
    
    occupancy_signal = Signal(
        state=[float(occupancy_data[i])]
    )
    ac_signal = Signal(
        state=[float(ac_data[i])]
    )
    purifier_signal = Signal(
        state=[float(purifier_data[i])]
    )
    
    future = ac_event.publish(ac_signal)
    _ = future.get(timeout=60)
    
    future = aqi_event.publish(aqi_signal)
    _ = future.get(timeout=60)
    
    future = occupancy_event.publish(occupancy_signal)
    _ = future.get(timeout=60)
    
    future = purifier_event.publish(purifier_signal)
    _ = future.get(timeout=60)
    
    future = temperature_event.publish(temp_signal)
    _ = future.get(timeout=60)
    
    time.sleep(1)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:35<00:00,  1.03s/it]


### Next state predictions for an ESS

In [4]:
eca_server = os.getenv("ECA_SERVER")

config = Config(uri=eca_server, username=username, password=password)
eca = ECA(config)


In [5]:
res, status = eca.ess.search(data=Search(
    module_id="temperature",
    state=[25.0],
    limit=1
))
prettyprint(res, status)

[SearchResult(score=0.0, ess=ESSResponse(id=5, state=[25.0], module_id='temperature', valence=-90.0, created_at='2023-05-31T13:42:14.404492Z', updated_at='2023-05-31T13:52:16.336662Z', avg_ess_valence=0.0, total_ess_score=0, avg_ess_score=0, score=2, embedding_input=None, aggregate_id=[], aggregate_valence=[], aggregate_score=[], aggregate_module_id=[], aggregate_shape=[], aggregate_context=[], aggregate_emb_inp=[], context=None))]
200


In [7]:
nsr = NextStateRequest(
    module_id="temperature",
    recall=3,
    range=2,
    n=3,
    status='n',
    current_state=5,
    previous_state=[4, 7]
)
res, status = eca.cortex.predictions_raw(data=nsr)
prettyprint(res, status)

{'current_state': [{'aggregate_id': 0,
                    'ess_id': 5,
                    'module_id': 'temperature',
                    'state': [25.0]}],
 'next_state': [{'list': [{'aggregate_id': 0,
                           'ess_id': 3,
                           'module_id': 'temperature',
                           'state': [24.0]}]},
                {'list': [{'aggregate_id': 0,
                           'ess_id': 4,
                           'module_id': 'temperature',
                           'state': [23.1]}]},
                {'list': [{'aggregate_id': 0,
                           'ess_id': 6,
                           'module_id': 'temperature',
                           'state': [23.0]}]}]}
200


### Next state predictions for an Aggregate

In [10]:
res, status = eca.ess.search(data=Search(
    module_id="purifier",
    state=[2.0],
    limit=1
))
prettyprint(res, status)

[SearchResult(score=0.0, ess=ESSResponse(id=3, state=[2.0], module_id='purifier', valence=90.0, created_at='2023-05-31T13:42:14.619965Z', updated_at='2023-05-31T13:52:30.166527Z', avg_ess_valence=0.0, total_ess_score=0, avg_ess_score=0, score=13, embedding_input=None, aggregate_id=[], aggregate_valence=[], aggregate_score=[], aggregate_module_id=[], aggregate_shape=[], aggregate_context=[], aggregate_emb_inp=[], context=None))]
200


In [11]:
ess_air_conditioner = ESSResponse(
    id=5,
    state=[25.0],
    module_id='air_conditioner',
    valence=90.0,
    score=8,
)
ess_occupancy = ESSResponse(
    id=7,
    state=[10.0],
    module_id='occupancy',
    valence=-90.0,
    score=3,
)
ess_purifier = ESSResponse(
    id=3,
    state=[2.0],
    module_id='purifier',
    valence=90.0,
    score=13,
)

In [13]:
res, status = eca.ess.search(data=Search(
    module_id="temperature",
    state=[29.0],
    limit=1
))
prettyprint(res, status)

[SearchResult(score=1.0, ess=ESSResponse(id=2, state=[28.0], module_id='temperature', valence=-90.0, created_at='2023-05-31T13:42:14.357678Z', updated_at='2023-05-31T13:51:56.816676Z', avg_ess_valence=0.0, total_ess_score=0, avg_ess_score=0, score=1, embedding_input=None, aggregate_id=[], aggregate_valence=[], aggregate_score=[], aggregate_module_id=[], aggregate_shape=[], aggregate_context=[], aggregate_emb_inp=[], context=None))]
200


In [14]:
ess_in = ESSInsert(
    module_id="air_quality_index",
    state=[90.0],
    valence=-90,
    score=1,
    context=None,
    embedding_input=None
)
ess_air_quality_index, status = eca.ess.create(data=ess_in)
prettyprint(ess_air_quality_index, status)

ess_in = ESSInsert(
    module_id="temperature",
    state=[29.0],
    valence=-90,
    score=1,
    context=None,
    embedding_input=None
)
ess_temperature, status = eca.ess.create(data=ess_in)
prettyprint(ess_temperature, status)

{'aggregate_context': [],
 'aggregate_emb_inp': [],
 'aggregate_id': [],
 'aggregate_module_id': [],
 'aggregate_score': [],
 'aggregate_shape': [],
 'aggregate_valence': [],
 'avg_ess_score': 0,
 'avg_ess_valence': 0.0,
 'context': None,
 'created_at': '2023-05-31T14:00:46.240287Z',
 'embedding_input': None,
 'id': 18,
 'module_id': 'air_quality_index',
 'score': 1,
 'state': [90.0],
 'total_ess_score': 0,
 'updated_at': '2023-05-31T14:00:46.240287Z',
 'valence': -90.0}
201
{'aggregate_context': [],
 'aggregate_emb_inp': [],
 'aggregate_id': [],
 'aggregate_module_id': [],
 'aggregate_score': [],
 'aggregate_shape': [],
 'aggregate_valence': [],
 'avg_ess_score': 0,
 'avg_ess_valence': 0.0,
 'context': None,
 'created_at': '2023-05-31T14:00:46.268653Z',
 'embedding_input': None,
 'id': 13,
 'module_id': 'temperature',
 'score': 1,
 'state': [29.0],
 'total_ess_score': 0,
 'updated_at': '2023-05-31T14:00:46.268653Z',
 'valence': -90.0}
201


In [15]:
aggregate_insert = ESSInsert(
    module_id="aggregate001",
    state=[],
    valence=0.0, 
)
agg_in = create_aggregate_from_ess(
    [
        ess_air_conditioner,
        ess_air_quality_index,
        ess_occupancy,
        ess_purifier,
        ess_temperature
    ],
    aggregate_insert)
res, status = eca.ess.create(data=agg_in)
prettyprint(res, status)

{'aggregate_context': [None, None, None, None, None],
 'aggregate_emb_inp': [None, None, None, None, None],
 'aggregate_id': [5, 18, 7, 3, 13],
 'aggregate_module_id': ['air_conditioner',
                         'air_quality_index',
                         'occupancy',
                         'purifier',
                         'temperature'],
 'aggregate_score': [8, 1, 3, 13, 1],
 'aggregate_shape': [1, 1, 1, 1, 1],
 'aggregate_valence': [90.0, -90.0, -90.0, 90.0, -90.0],
 'avg_ess_score': 5,
 'avg_ess_valence': -18.0,
 'context': None,
 'created_at': '2023-05-31T14:01:18.009593Z',
 'embedding_input': None,
 'id': 33,
 'module_id': 'aggregate001',
 'score': 0,
 'state': [25.0, 90.0, 10.0, 2.0, 29.0],
 'total_ess_score': 26,
 'updated_at': '2023-05-31T14:01:18.009593Z',
 'valence': -90.0}
201


In [17]:
nsr = NextStateRequest(
    module_id="aggregate001",
    recall=3,
    range=2,
    n=3,
    status='n',
    current_state=33,
    previous_state=[20, 25]
)
res, status = eca.cortex.predictions_raw(data=nsr)
prettyprint(res, status)

{'current_state': [{'aggregate_id': 33,
                    'ess_id': 5,
                    'module_id': 'air_conditioner',
                    'state': [25.0]},
                   {'aggregate_id': 33,
                    'ess_id': 18,
                    'module_id': 'air_quality_index',
                    'state': [90.0]},
                   {'aggregate_id': 33,
                    'ess_id': 7,
                    'module_id': 'occupancy',
                    'state': [10.0]},
                   {'aggregate_id': 33,
                    'ess_id': 3,
                    'module_id': 'purifier',
                    'state': [2.0]},
                   {'aggregate_id': 33,
                    'ess_id': 13,
                    'module_id': 'temperature',
                    'state': [29.0]}],
 'next_state': [{'list': [{'aggregate_id': 6,
                           'ess_id': 3,
                           'module_id': 'air_conditioner',
                           'state': [18.0]},
        

### Natural Language Retrieval

In [22]:
retrieval = RetrievalRequest(
    query="what is the temperature and the air quality index on the 31st of may at around 1:50pm?"
)
res, status = eca.retrieval.query(data=retrieval)
prettyprint(res, status)

{'response': 'The temperature on May 31st at around 1:50 pm was 22 degrees '
             'Celsius and the air quality index was 17.'}
200
